# Deskryptory i protokół atrybutów

Od właściwości prostych po kontrolę dostępu i cache na poziomie klasy.


## Cele
- zrozumieć protokół `__get__`, `__set__`, `__delete__`
- stworzyć deskryptor walidujący i pamiętający dane per instancja
- poznać `@property` jako uproszczony deskryptor


## `@property` jako deskryptor
Każde `property` to obiekt implementujący protokół deskryptora.


In [ ]:
class Temperature:
    def __init__(self, celsius: float):
        self._celsius = celsius

    @property
    def celsius(self) -> float:
        return self._celsius

    @celsius.setter
    def celsius(self, value: float):
        if value < -273.15:
            raise ValueError("Niemożliwa temperatura")
        self._celsius = value

room = Temperature(20.0)
room.celsius = 22
print(room.celsius)


## Własny deskryptor walidujący
Tworzymy deskryptor wielokrotnego użytku przechowujący dane w `__dict__` instancji.


In [ ]:
class PositiveNumber:
    def __init__(self, name: str):
        self.name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if value <= 0:
            raise ValueError("Wartość musi być dodatnia")
        instance.__dict__[self.name] = value

class Account:
    balance = PositiveNumber("balance")

    def __init__(self, balance: float):
        self.balance = balance

account = Account(100)
account.balance = 50
print(account.balance)


## Deskryptor obliczany z cache
`functools.cached_property` to przykład deskryptora obliczającego wartość raz.


In [ ]:
from functools import cached_property

class Matrix:
    def __init__(self, data):
        self.data = data

    @cached_property
    def determinant(self):
        # uproszczony przypadek dla macierzy 2x2
        a, b = self.data[0]
        c, d = self.data[1]
        print("Liczę wyznacznik...")
        return a * d - b * c

m = Matrix([[1, 2], [3, 4]])
print(m.determinant)
print(m.determinant)


**Podsumowanie:** Deskryptory umożliwiają ponowne wykorzystanie logiki zarządzania atrybutami.

**Pytanie kontrolne:** Jakie korzyści daje przechowywanie danych w `instance.__dict__` zamiast atrybutu deskryptora?


### 🧩 Zadanie 1
Zaprojektuj deskryptor `Email`, który waliduje format adresu e-mail i przechowuje go w instancji.


In [ ]:
# Rozwiązanie Zadania 1
import re

class Email:
    pattern = re.compile(r"^[^@]+@[^@]+\.[^@]+$")

    def __init__(self, name: str):
        self.name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__.get(self.name)

    def __set__(self, instance, value):
        if not self.pattern.match(value):
            raise ValueError("Niepoprawny email")
        instance.__dict__[self.name] = value

class User:
    email = Email("email")

    def __init__(self, email):
        self.email = email

u = User("adam@example.com")
print(u.email)


### 🧩 Zadanie 2
Stwórz deskryptor `HistoryField`, który zapisuje historię zmian wartości w liście.


In [1]:
# Rozwiązanie Zadania 2
class HistoryField:
    def __init__(self, name: str):
        self.name = name
        self.history_name = f"_{name}_history"

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__.get(self.name)

    def __set__(self, instance, value):
        history = instance.__dict__.setdefault(self.history_name, [])
        history.append(value)
        instance.__dict__[self.name] = value

    def history(self, instance):
        return instance.__dict__.get(self.history_name, [])

class Product:
    price = HistoryField("price")

    def __init__(self, price):
        self.price = price

item = Product(10)
item.price = 12
print(item.price)
print(Product.price.history(item))


12
[10, 12]
